Workflow followed according to: https://gatk.broadinstitute.org/hc/en-us/articles/360035531192-RNAseq-short-variant-discovery-SNPs-Indels

Github workflow for GATK4 here: https://github.com/gatk-workflows/gatk4-rnaseq-germline-snps-indels/blob/master/gatk4-rna-best-practices.wdl

In [6]:
import os
import pysam
import pandas as pd
from tqdm import tqdm
from varseek.varseek_build import add_mutation_type
from varseek.utils import convert_chromosome_value_to_int_when_possible, calculate_metrics, draw_confusion_matrix, create_venn_diagram, calculate_sensitivity_specificity, create_stratified_metric_bar_plot
from varseek.constants import complement

In [7]:
import random

def generate_random_fastq(output_path, num_sequences=5000, seq_length=150, quality_score="I"):
    if not os.path.exists(output_path):
        if not os.path.exists(os.path.dirname(output_path)):
            os.makedirs(os.path.dirname(output_path))
        bases = ['A', 'C', 'G', 'T']
        with open(output_path, "w") as fastq_file:
            for i in range(num_sequences):
                # Generate a random sequence of the specified length
                sequence = ''.join(random.choices(bases, k=seq_length))
                
                # Create a quality string of the same length, filled with the specified quality score
                quality = quality_score * seq_length
                
                # Write the FASTQ entry
                fastq_file.write(f"@seq_{i + 1}\n")
                fastq_file.write(f"{sequence}\n")
                fastq_file.write("+\n")
                fastq_file.write(f"{quality}\n")

In [8]:
synthetic_read_fastq = "/home/jrich/data/varseek_data/synthetic_data/synthetic_reads.fastq"  #!!! update path
unique_mcrs_df_path = "/home/jrich/data/varseek_data/unique_mcrs.csv"  #!!! update path
generate_random_fastq(synthetic_read_fastq)  #!!! ERASE AND REPLACE THE ABOVE PATH WITH REAL DATA

out_dir_base = "/home/jrich/data/varseek_data"
threads = 32
read_length = 150
mutation_source = "cdna"  # "cdna", "cds"

cosmic_tsv = "/home/jrich/data/varseek_data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37.tsv"
cosmic_cdna_info_csv = "/home/jrich/data/varseek_data/reference/cosmic/CancerMutationCensus_AllData_Tsv_v100_GRCh37/CancerMutationCensus_AllData_v100_GRCh37_mutation_workflow_with_cdna.csv"

# if these paths don't exist then they will be created
reference_genome_fasta = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa"
reference_genome_gtf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf"
genomes1000_vcf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf"
ensembl_germline_vcf = "/home/jrich/data/varseek_data/reference/ensembl_grch37_release93/homo_sapiens.vcf"

STAR = "/home/jrich/opt/STAR-2.7.11b/source/STAR"
java = "/home/jrich/opt/java/jdk-17.0.12+7/bin/java"
picard_jar = "/home/jrich/opt/picard/build/libs/picard.jar"
gatk = "/home/jrich/opt/gatk-4.6.0.0/gatk"

In [9]:
gatk_parent = f"{out_dir_base}/gatk"

genome_dir = f"{gatk_parent}/reference"
os.makedirs(genome_dir, exist_ok=True)

alignment_folder = f"{gatk_parent}/alignment"
os.makedirs(alignment_folder, exist_ok=True)

gatk_supporting_files = f"{gatk_parent}/supporting_files"
os.makedirs(alignment_folder, exist_ok=True)

ensembl_germline_vcf_filtered = ensembl_germline_vcf.replace(".vcf", "_filtered.vcf")

out_file_name_prefix = f"{alignment_folder}/sample_"

vcf_folder = f"{gatk_parent}/vcfs"
haplotypecaller_folder = f"{vcf_folder}/haplotypecaller"
mutect2_folder = f"{vcf_folder}/mutect2"

os.makedirs(vcf_folder, exist_ok=True)
os.makedirs(haplotypecaller_folder, exist_ok=True)
os.makedirs(mutect2_folder, exist_ok=True)

aligned_and_unmapped_bam = f"{out_file_name_prefix}Aligned.sortedByCoord.out.bam"
aligned_only_bam = f"{alignment_folder}/aligned_only.bam"
unmapped_bam = f"{alignment_folder}/unmapped.bam"
merged_bam = f"{alignment_folder}/merged.bam"

marked_duplicates_bam = f"{alignment_folder}/marked_duplicates.bam"
marked_dup_metrics_txt = f"{alignment_folder}/marked_dup_metrics.txt"

split_n_cigar_reads_bam = f"{alignment_folder}/split_n_cigar_reads.bam"
recal_data_table = f"{alignment_folder}/recal_data.table"
recalibrated_bam = f"{alignment_folder}/recalibrated.bam"
covariates_plot = f"{alignment_folder}/AnalyzeCovariates.pdf"
haplotypecaller_unfiltered_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_unfiltered.g.vcf.gz"

haplotypecaller_filtered_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_filtered.vcf.gz"
haplotypecaller_filtered_applied_vcf = f"{haplotypecaller_folder}/haplotypecaller_output_filtered_applied.vcf.gz"

panel_of_normals_vcf = f"{gatk_supporting_files}/1000g_pon.hg38.vcf.gz"
mutect2_unfiltered_vcf = f"{mutect2_folder}/mutect2_output_unfiltered.g.vcf.gz"
mutect2_filtered_vcf = f"{mutect2_folder}/mutect2_output_filtered.vcf.gz"

In [19]:
if not os.path.exists(reference_genome_fasta):
    !wget -O {reference_genome_fasta}.gz https://ftp.ensembl.org/pub/grch37/release-93/fasta/homo_sapiens/dna/Homo_sapiens.GRCh37.dna.primary_assembly.fa.gz && gunzip {reference_genome_fasta}.gz

if not os.path.exists(reference_genome_gtf):
    !wget -O {reference_genome_gtf}.gz https://ftp.ensembl.org/pub/grch37/release-93/gtf/homo_sapiens/Homo_sapiens.GRCh37.87.gtf.gz && gunzip {reference_genome_gtf}.gz

if not os.path.exists(genomes1000_vcf):
    !wget -O {genomes1000_vcf}.gz https://ftp.ensembl.org/pub/grch37/release-93/variation/vcf/homo_sapiens/1000GENOMES-phase_3.vcf.gz && gunzip {genomes1000_vcf}.gz
    
if not os.path.exists(ensembl_germline_vcf):
    !wget -O {ensembl_germline_vcf}.gz https://ftp.ensembl.org/pub/grch37/release-93/variation/vcf/homo_sapiens/homo_sapiens.vcf.gz && gunzip {ensembl_germline_vcf}.gz

if not os.path.exists(ensembl_germline_vcf_filtered):
    # !awk 'BEGIN { FS = "\t" } $4 !~ /W/ && $5 !~ /W/' $ensembl_germline_vcf > $ensembl_germline_vcf_filtered
    !grep -vP '^[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*W|^[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*\t[^\t]*W' $ensembl_germline_vcf > $ensembl_germline_vcf_filtered

if not os.path.exists(panel_of_normals_vcf):
    !wget https://storage.googleapis.com/gatk-best-practices/somatic-hg38/1000g_pon.hg38.vcf.gz {gatk_supporting_files}
    !wget https://storage.googleapis.com/gatk-best-practices/somatic-hg38/1000g_pon.hg38.vcf.gz.tbi {gatk_supporting_files}

1. Mapping to the Reference with STAR

In [20]:
read_length_minus_one = read_length - 1

!$STAR \
    --runThreadN $threads \
    --runMode genomeGenerate \
    --genomeDir $genome_dir \
    --genomeFastaFiles $reference_genome_fasta \
    --sjdbGTFfile $reference_genome_gtf \
    --sjdbOverhang $read_length_minus_one

	/home/jrich/opt/STAR-2.7.11b/source/STAR --runThreadN 32 --runMode genomeGenerate --genomeDir /home/jrich/data/varseek_data/gatk/reference --genomeFastaFiles /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --sjdbGTFfile /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.87.gtf --sjdbOverhang 149
	STAR version: 2.7.11b   compiled: 2024-07-22T09:22:47-0700 dator:/home/jrich/opt/STAR-2.7.11b/source
Oct 30 09:24:08 ..... started STAR run
Oct 30 09:24:08 ... starting to generate Genome files
Oct 30 09:25:00 ..... processing annotations GTF
Oct 30 09:25:25 ... starting to sort Suffix Array. This may take a long time...
Oct 30 09:25:42 ... sorting Suffix Array chunks and saving them to disk...
Oct 30 09:33:31 ... loading chunks from disk, packing SA...
Oct 30 09:34:55 ... finished generating suffix array
Oct 30 09:34:55 ... generating Suffix Array index
Oct 30 09:38:55 ... completed Suffix Array in

In [21]:
#* --outSAMmapqUnique 60 - change from default of 255 to avoid colliding with some aligners' use of 255 as a special value

!$STAR \
    --runThreadN $threads \
    --genomeDir $genome_dir \
    --readFilesIn $synthetic_read_fastq \
    --sjdbOverhang $read_length_minus_one \
    --outFileNamePrefix $out_file_name_prefix \
    --outSAMtype BAM SortedByCoordinate \
    --outSAMunmapped Within \
    --outSAMmapqUnique 60 \
    --twopassMode Basic



	/home/jrich/opt/STAR-2.7.11b/source/STAR --runThreadN 32 --genomeDir /home/jrich/data/varseek_data/gatk/reference --readFilesIn /home/jrich/data/varseek_data/synthetic_data/synthetic_reads.fastq --sjdbOverhang 149 --outFileNamePrefix /home/jrich/data/varseek_data/gatk/alignment/sample_ --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outSAMmapqUnique 60 --twopassMode Basic
	STAR version: 2.7.11b   compiled: 2024-07-22T09:22:47-0700 dator:/home/jrich/opt/STAR-2.7.11b/source
Oct 30 09:43:02 ..... started STAR run
Oct 30 09:43:02 ..... loading genome
Oct 30 09:43:24 ..... started 1st pass mapping
Oct 30 09:43:28 ..... finished 1st pass mapping
Oct 30 09:43:29 ..... inserting junctions into the genome indices
Oct 30 09:45:04 ..... started mapping
Oct 30 09:45:10 ..... finished mapping
Oct 30 09:45:14 ..... started sorting BAM
Oct 30 09:45:14 ..... finished successfully


Separate unmapped reads into its own BAM file

In [22]:
# with pysam.AlignmentFile(aligned_and_unmapped_bam, "rb") as bam_in: 
#     # unmapped_header = bam_in.header.to_dict()
#     # if 'PG' in unmapped_header:
#     #     del unmapped_header['PG']
           
#     with pysam.AlignmentFile(aligned_only_bam, "wb", template=bam_in) as bam_aligned_out, pysam.AlignmentFile(unmapped_bam, "wb", template=bam_in) as bam_unmapped_out:
#         for read in bam_in:
#             if read.is_unmapped:
#                 bam_unmapped_out.write(read)  # Write unmapped read to the unmapped BAM file
#             else:
#                 bam_aligned_out.write(read)  # Write aligned read to the aligned BAM file

# print(f"Unmapped reads written to {unmapped_bam}")

#

In [23]:
!$java -jar $picard_jar FastqToSam \
    -FASTQ $synthetic_read_fastq \
    -OUTPUT $unmapped_bam \
    -READ_GROUP_NAME rg1 \
    -SAMPLE_NAME sample1 \
    -LIBRARY_NAME lib1 \
    -PLATFORM_UNIT unit1 \
    -PLATFORM ILLUMINA \
    -SEQUENCING_CENTER center1

09:45:17.964 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Oct 30 09:45:18 PDT 2024] FastqToSam --FASTQ /home/jrich/data/varseek_data/synthetic_data/synthetic_reads.fastq --OUTPUT /home/jrich/data/varseek_data/gatk/alignment/unmapped.bam --READ_GROUP_NAME rg1 --SAMPLE_NAME sample1 --LIBRARY_NAME lib1 --PLATFORM_UNIT unit1 --PLATFORM ILLUMINA --SEQUENCING_CENTER center1 --USE_SEQUENTIAL_FASTQS false --SORT_ORDER queryname --MIN_Q 0 --MAX_Q 93 --STRIP_UNPAIRED_MATE_NUMBER false --ALLOW_AND_IGNORE_EMPTY_LINES false --ALLOW_EMPTY_FASTQ false --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 5 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Wed Oct 30 09:45:18 PDT 2024] Executing as jrich@dator on Linux 3.10.0-1127.13.1

2. MergeBamAlignment

In [24]:
reference_genome_dict = reference_genome_fasta.replace(".fa", ".dict")

!$java -jar $picard_jar CreateSequenceDictionary \
    -R $reference_genome_fasta \
    -O $reference_genome_dict

09:45:19.462 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Oct 30 09:45:19 PDT 2024] CreateSequenceDictionary --OUTPUT /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.dict --REFERENCE /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --TRUNCATE_NAMES_AT_WHITESPACE true --NUM_SEQUENCES 2147483647 --VERBOSITY INFO --QUIET false --VALIDATION_STRINGENCY STRICT --COMPRESSION_LEVEL 5 --MAX_RECORDS_IN_RAM 500000 --CREATE_INDEX false --CREATE_MD5_FILE false --help false --version false --showHidden false --USE_JDK_DEFLATER false --USE_JDK_INFLATER false
[Wed Oct 30 09:45:19 PDT 2024] Executing as jrich@dator on Linux 3.10.0-1127.13.1.el7.x86_64 amd64; OpenJDK 64-Bit Server VM 17.0.12+7; Deflater: Intel; Inflater: Intel; Provider GCS is available; Picard version

In [25]:
!$java -jar $picard_jar MergeBamAlignment \
    --ALIGNED_BAM $aligned_and_unmapped_bam \
    --UNMAPPED_BAM $unmapped_bam \
    --OUTPUT $merged_bam \
    --REFERENCE_SEQUENCE $reference_genome_fasta \
    --SORT_ORDER coordinate \
    --INCLUDE_SECONDARY_ALIGNMENTS false \
    --VALIDATION_STRINGENCY SILENT



09:45:37.369 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Oct 30 09:45:37 PDT 2024] MergeBamAlignment --UNMAPPED_BAM /home/jrich/data/varseek_data/gatk/alignment/unmapped.bam --ALIGNED_BAM /home/jrich/data/varseek_data/gatk/alignment/sample_Aligned.sortedByCoord.out.bam --OUTPUT /home/jrich/data/varseek_data/gatk/alignment/merged.bam --SORT_ORDER coordinate --INCLUDE_SECONDARY_ALIGNMENTS false --VALIDATION_STRINGENCY SILENT --REFERENCE_SEQUENCE /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --ADD_PG_TAG_TO_READS true --PAIRED_RUN true --CLIP_ADAPTERS true --IS_BISULFITE_SEQUENCE false --ALIGNED_READS_ONLY false --MAX_INSERTIONS_OR_DELETIONS 1 --ATTRIBUTES_TO_REVERSE OQ --ATTRIBUTES_TO_REVERSE U2 --ATTRIBUTES_TO_REVERSE_COMPLEMENT E2 --ATTRIBUTES_TO_REVERSE_COMPLEMENT SQ --READ1_TRIM 0 --READ2_TRIM 0 --ALIG

3. MarkDuplicates

In [26]:
!$java -jar $picard_jar MarkDuplicates \
      --INPUT $merged_bam \
      --OUTPUT $marked_duplicates_bam \
      --METRICS_FILE $marked_dup_metrics_txt \
      --CREATE_INDEX true \
      --VALIDATION_STRINGENCY SILENT

09:45:38.909 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/picard/build/libs/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Oct 30 09:45:38 PDT 2024] MarkDuplicates --INPUT /home/jrich/data/varseek_data/gatk/alignment/merged.bam --OUTPUT /home/jrich/data/varseek_data/gatk/alignment/marked_duplicates.bam --METRICS_FILE /home/jrich/data/varseek_data/gatk/alignment/marked_dup_metrics.txt --VALIDATION_STRINGENCY SILENT --CREATE_INDEX true --MAX_SEQUENCES_FOR_DISK_READ_ENDS_MAP 50000 --MAX_FILE_HANDLES_FOR_READ_ENDS_MAP 8000 --SORTING_COLLECTION_SIZE_RATIO 0.25 --TAG_DUPLICATE_SET_MEMBERS false --REMOVE_SEQUENCING_DUPLICATES false --TAGGING_POLICY DontTag --CLEAR_DT true --DUPLEX_UMI false --FLOW_MODE false --FLOW_DUP_STRATEGY FLOW_QUALITY_SUM_STRATEGY --USE_END_IN_UNPAIRED_READS false --USE_UNPAIRED_CLIPPED_END false --UNPAIRED_END_UNCERTAINTY 0 --UNPAIRED_START_UNCERTAINTY 0 --FLOW_SKIP_FIRST_N_FLOWS 0 --FLOW_Q_IS_KNOWN_END false --FL

4. SplitNCigarReads

In [44]:
_ = pysam.faidx(reference_genome_fasta)

!$gatk SplitNCigarReads \
    -R $reference_genome_fasta \
    -I $marked_duplicates_bam \
    -O $split_n_cigar_reads_bam

4221.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar SplitNCigarReads -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk/alignment/marked_duplicates.bam -O /home/jrich/data/varseek_data/gatk/alignment/split_n_cigar_reads.bam
10:09:19.678 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:09:19.869 INFO  SplitNCigarReads - ------------------------------------------------------------
10:09:19.875 INFO  SplitNCigarReads - The Genome Analysis Toolkit (GATK) v4.6.0.0
10:09:19.875 INFO  SplitNCigarReads - For supp

5. BaseRecalibrator

In [28]:
# -I in old version, -F in new version
!$gatk IndexFeatureFile \
    -I $ensembl_germline_vcf_filtered

!$gatk IndexFeatureFile \
    -I $genomes1000_vcf

Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar IndexFeatureFile -I /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/homo_sapiens_filtered.vcf
09:45:53.197 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
09:45:53.343 INFO  IndexFeatureFile - ------------------------------------------------------------
09:45:53.348 INFO  IndexFeatureFile - The Genome Analysis Toolkit (GATK) v4.6.0.0
09:45:53.348 INFO  IndexFeatureFile - For support and documentation go to https://software.broadinstitute.org/gatk/
09:45:53.348 INFO  IndexFeatureFile - Executing as jrich@dator on Linux v3.10.0-1127.13.1.

In [45]:
!$gatk BaseRecalibrator \
    -I $split_n_cigar_reads_bam \
    -R $reference_genome_fasta \
    --use-original-qualities \
    --known-sites $ensembl_germline_vcf_filtered \
    --known-sites $genomes1000_vcf \
    -O $recal_data_table

# -known-sites ${dbSNP_vcf} \
# -known-sites ${sep=" --known-sites " known_indels_sites_VCFs}



4267.43s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar BaseRecalibrator -I /home/jrich/data/varseek_data/gatk/alignment/split_n_cigar_reads.bam -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa --use-original-qualities --known-sites /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/homo_sapiens_filtered.vcf --known-sites /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/1000GENOMES-phase_3.vcf -O /home/jrich/data/varseek_data/gatk/alignment/recal_data.table
10:10:05.471 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so

6. Apply Recalibration

In [46]:
!$gatk ApplyBQSR \
    --add-output-sam-program-record \
    -R $reference_genome_fasta \
    -I $split_n_cigar_reads_bam \
    --use-original-qualities \
    --bqsr-recal-file $recal_data_table \
    -O $recalibrated_bam



4302.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar ApplyBQSR --add-output-sam-program-record -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk/alignment/split_n_cigar_reads.bam --use-original-qualities --bqsr-recal-file /home/jrich/data/varseek_data/gatk/alignment/recal_data.table -O /home/jrich/data/varseek_data/gatk/alignment/recalibrated.bam
10:10:40.522 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:10:40.666 INFO  ApplyBQSR - ------------------------------------------------------------
10:10:40.6

7. AnalyzeCovariates

In [31]:
!$gatk AnalyzeCovariates \
    -bqsr $recal_data_table \
    -plots $covariates_plot



Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar AnalyzeCovariates -bqsr /home/jrich/data/varseek_data/gatk/alignment/recal_data.table -plots /home/jrich/data/varseek_data/gatk/alignment/AnalyzeCovariates.pdf
09:51:28.874 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
09:51:29.024 INFO  AnalyzeCovariates - ------------------------------------------------------------
09:51:29.028 INFO  AnalyzeCovariates - The Genome Analysis Toolkit (GATK) v4.6.0.0
09:51:29.029 INFO  AnalyzeCovariates - For support and documentation go to https://software.broadinstitute.org/gatk/
09:51:29.029 INFO  AnalyzeCovariates -

8a. HaplotypeCaller

In [49]:
!$gatk --java-options "-Xmx4g" HaplotypeCaller  \
    -R $reference_genome_fasta \
    -I $recalibrated_bam \
    -O $haplotypecaller_unfiltered_vcf \
    --standard-min-confidence-threshold-for-calling 20 \
    -dont-use-soft-clipped-bases \



4384.11s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -Xmx4g -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar HaplotypeCaller -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk/alignment/recalibrated.bam -O /home/jrich/data/varseek_data/gatk/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz --standard-min-confidence-threshold-for-calling 20 -dont-use-soft-clipped-bases
10:12:02.609 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:12:02.793 INFO  HaplotypeCaller - ------------------------------------------------------------
10:12:02.798 INFO 

8.5a. MergeVcfs

In [33]:
# merged_vcf = haplotypecaller_merged.vcf
# 
# !$java -jar $picard_jar MergeVcfs \
#     --INPUT $vcf1 \
#     --INPUT $vcf2 \
#     --OUTPUT $merged_vcf

9a. VariantFiltration

In [61]:
# cosmic_vcf = ""

!$gatk VariantFiltration \
    -R $reference_genome_fasta \
    -V $haplotypecaller_unfiltered_vcf \
    -O $haplotypecaller_filtered_vcf \
    --window 35 \
    --cluster 3 \
    --filter-name "FS" \
    --filter "FS > 30.0" \
    --filter-name "QD" \
    --filter "QD < 2.0"

    # --mask $cosmic_vcf \
    # --mask-name "COSMIC" \
    # --filter-not-in-mask \



5675.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar VariantFiltration -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -V /home/jrich/data/varseek_data/gatk/vcfs/haplotypecaller/haplotypecaller_output_unfiltered.g.vcf.gz -O /home/jrich/data/varseek_data/gatk/vcfs/haplotypecaller/haplotypecaller_output_filtered.vcf.gz --window 35 --cluster 3 --filter-name FS --filter FS > 30.0 --filter-name QD --filter QD < 2.0
10:33:33.819 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:33:33.966 INFO  VariantFiltration - ----------------------------------

10a. Do the filtering

In [ ]:
!$gatk SelectVariants \
     -V $haplotypecaller_filtered_vcf \
     --exclude-filtered true \
     -O $haplotypecaller_filtered_applied_vcf

8b. Mutect2

In [50]:
# consider adding --disable-read-filter
!$gatk Mutect2 \
    -R $reference_genome_fasta \
    -I $recalibrated_bam \
    -O $mutect2_unfiltered_vcf \
    --panel-of-normals $panel_of_normals_vcf \
    --min-base-quality-score 20



4850.19s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar Mutect2 -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -I /home/jrich/data/varseek_data/gatk/alignment/recalibrated.bam -O /home/jrich/data/varseek_data/gatk/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz --min-base-quality-score 20
10:19:48.078 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:19:48.217 INFO  Mutect2 - ------------------------------------------------------------
10:19:48.222 INFO  Mutect2 - The Genome Analysis Toolkit (GATK) v4.6.0.0
10:19:48.222 INFO  Mutect2 - For sup

9b. FilterMutectCalls

In [62]:
# # if multiple stats files:
# !$gatk MergeMutectStats -stats unfiltered1.vcf.stats -stats unfiltered2.vcf.stats -O unfiltered.vcf.stats

# stats_file = f"{mutect2_unfiltered_vcf}.stats"
# --stats $stats_file
!$gatk FilterMutectCalls \
    -R $reference_genome_fasta \
    -V $mutect2_unfiltered_vcf \
    -O $mutect2_filtered_vcf



5731.66s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using GATK jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar FilterMutectCalls -R /home/jrich/data/varseek_data/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.dna.primary_assembly.fa -V /home/jrich/data/varseek_data/gatk/vcfs/mutect2/mutect2_output_unfiltered.g.vcf.gz -O /home/jrich/data/varseek_data/gatk/vcfs/mutect2/mutect2_output_filtered.vcf.gz
10:34:30.141 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/jrich/opt/gatk-4.6.0.0/gatk-package-4.6.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
10:34:30.387 INFO  FilterMutectCalls - ------------------------------------------------------------
10:34:30.392 INFO  FilterMutectCalls - The Genome Analysis Toolkit (GATK) v4.6.0.0
10:34:30.393 INFO

In [74]:
# def create_sample_vcf(output_path):
#     with open(output_path, "w") as vcf_file:
#         # Write VCF headers
#         vcf_file.write("##fileformat=VCFv4.2\n")
#         vcf_file.write("##source=SampleVCFGenerator\n")
#         vcf_file.write("##reference=GRCh37\n")
#         vcf_file.write("##INFO=<ID=DP,Number=1,Type=Integer,Description=\"Total Depth\">\n")
#         vcf_file.write("##INFO=<ID=AF,Number=A,Type=Float,Description=\"Allele Frequency\">\n")
#         vcf_file.write("##FILTER=<ID=PASS,Description=\"All filters passed\">\n")
#         vcf_file.write("##FORMAT=<ID=GT,Number=1,Type=String,Description=\"Genotype\">\n")
#         vcf_file.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tsample1\n")
        
#         # Write sample variant entries
#         variants = [
#             ("1", 123456, ".", "G", "A", 50, "PASS", "DP=100;AF=0.5", "GT", "0/1"),
#             ("1", 234567, ".", "C", "T", 60, "PASS", "DP=200;AF=0.3", "GT", "1/1"),
#             ("2", 345678, ".", "T", "G", 70, "PASS", "DP=150;AF=0.1", "GT", "0/1"),
#             ("X", 456789, ".", "A", "C", 80, "PASS", "DP=120;AF=0.05", "GT", "0/0")
#         ]
        
#         for chrom, pos, var_id, ref, alt, qual, fltr, info, fmt, sample in variants:
#             vcf_file.write(f"{chrom}\t{pos}\t{var_id}\t{ref}\t{alt}\t{qual}\t{fltr}\t{info}\t{fmt}\t{sample}\n")

# # Usage
# create_sample_vcf("/home/jrich/data/varseek_data/gatk/sample.vcf")

# df_sample = vcf_to_dataframe("/home/jrich/data/varseek_data/gatk/sample.vcf")

[W::vcf_parse] Contig '1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '2' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'X' is not defined in the header. (Quick workaround: index the file with tabix.)


In [73]:
# convert gatk output vcf to pandas df
def vcf_to_dataframe(vcf_file):
    """Convert a VCF file to a Pandas DataFrame."""
    vcf = pysam.VariantFile(vcf_file)
    
    # List to store VCF rows
    vcf_data = []
    
    # Fetch each record in the VCF
    for record in vcf.fetch():
        # For each record, extract the desired fields
        vcf_row = {
            'CHROM': record.chrom,
            'POS': record.pos,
            'ID': record.id,
            'REF': record.ref,
            'ALT': ','.join(record.alts),  # ALT can be multiple
        }
        
        # Append the row to the list
        vcf_data.append(vcf_row)
    
    # Convert the list to a Pandas DataFrame
    df = pd.DataFrame(vcf_data)
    
    return df

# Convert VCF to DataFrame
df_hap = vcf_to_dataframe(haplotypecaller_filtered_applied_vcf)
df_mut = vcf_to_dataframe(mutect2_filtered_vcf)



In [78]:
# load in COSMIC tsv with columns CHROM, POS, ID, REF, ALT
cosmic_df = pd.read_csv(cosmic_tsv, sep="\t", usecols=["Mutation genome position GRCh37", "GENOMIC_WT_ALLELE_SEQ", "GENOMIC_MUT_ALLELE_SEQ", "ACCESSION_NUMBER", "Mutation CDS", "MUTATION_URL"])

if mutation_source == "cdna":
    cosmic_cdna_info_df = pd.read_csv(cosmic_cdna_info_csv, usecols=["mutation_id", "mutation"])
    cosmic_cdna_info_df = cosmic_cdna_info_df.rename(columns={"mutation": "Mutation cDNA"})



In [84]:
cosmic_df = add_mutation_type(cosmic_df, "Mutation CDS")

cosmic_df["ACCESSION_NUMBER"] = cosmic_df["ACCESSION_NUMBER"].str.split(".").str[0]

cosmic_df[['CHROM', 'GENOME_POS']] = cosmic_df['Mutation genome position GRCh37'].str.split(':', expand=True)
# cosmic_df['CHROM'] = cosmic_df['CHROM'].apply(convert_chromosome_value_to_int_when_possible)
cosmic_df[['POS', 'GENOME_END_POS']] = cosmic_df['GENOME_POS'].str.split('-', expand=True)

cosmic_df = cosmic_df.rename(
    columns={
        "GENOMIC_WT_ALLELE_SEQ": "REF",
        "GENOMIC_MUT_ALLELE_SEQ": "ALT",
        "MUTATION_URL": "mutation_id"
    }
)

if mutation_source == "cds":
    cosmic_df['ID'] = cosmic_df['ACCESSION_NUMBER'] + ":" + cosmic_df['Mutation CDS']
elif mutation_source == "cdna":
    cosmic_df["mutation_id"] = cosmic_df["mutation_id"].str.extract(r"id=(\d+)")
    cosmic_df['mutation_id'] = cosmic_df['mutation_id'].astype(int, errors='raise')
    cosmic_df = cosmic_df.merge(cosmic_cdna_info_df[['mutation_id', 'Mutation cDNA']], on='mutation_id', how='left')
    cosmic_df['ID'] = cosmic_df['ACCESSION_NUMBER'] + ":" + cosmic_df['Mutation cDNA']
    cosmic_df.drop(columns=["Mutation cDNA"], inplace=True)

In [86]:
# # commented out because this was only used when considering strand below for reverse-complementing, but the line below is irrelevant now
# gtf_df = pd.read_csv(reference_genome_gtf, sep='\t', comment='#', header=None, names=[
# 'seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])

# gtf_df = gtf_df[gtf_df['feature'] == 'transcript']

# gtf_df['transcript_id'] = gtf_df['attribute'].str.extract('transcript_id "([^"]+)"')

# gtf_df = gtf_df.dropna(subset=['transcript_id'])

# gtf_df = gtf_df[['transcript_id', 'strand']].rename(
#     columns={'transcript_id': 'ACCESSION_NUMBER'})

# cosmic_df = pd.merge(cosmic_df, gtf_df, on='ACCESSION_NUMBER', how='left')
# cosmic_df['strand'] = cosmic_df['strand'].fillna('.')

/tmp/ipykernel_145075/2323330122.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  gtf_df = pd.read_csv(reference_genome_gtf, sep='\t', comment='#', header=None, names=[


In [88]:
cosmic_df = cosmic_df.dropna(subset=['CHROM', 'POS'])
cosmic_df = cosmic_df.dropna(subset=['ID'])  # a result of intron mutations and COSMIC duplicates that get dropped before cDNA determination

In [98]:
# reference_genome_fasta
reference_genome = pysam.FastaFile(reference_genome_fasta)

def get_nucleotide_from_reference(chromosome, position):
    # pysam is 0-based, so subtract 1 from the position
    return reference_genome.fetch(chromosome, int(position) - 1, int(position))

def get_complement(nucleotide_sequence):
    return ''.join([complement[nuc] for nuc in nucleotide_sequence])

# Insertion, get original nucleotide (not in COSMIC df)
cosmic_df.loc[
    (cosmic_df['GENOME_END_POS'].astype(int) != 1) & (cosmic_df['mutation_type'] == 'insertion'), 'original_nucleotide'
] = cosmic_df.loc[
    (cosmic_df['GENOME_END_POS'].astype(int) != 1) & (cosmic_df['mutation_type'] == 'insertion'), ['CHROM', 'POS']
].progress_apply(
    lambda row: get_nucleotide_from_reference(row['CHROM'], int(row['POS'])),
    axis=1
)

# Deletion, get new nucleotide (not in COSMIC df)
cosmic_df.loc[
    (cosmic_df['POS'].astype(int) != 1) & (cosmic_df['mutation_type'] == 'deletion'), 'original_nucleotide'
] = cosmic_df.loc[
    (cosmic_df['POS'].astype(int) != 1) & (cosmic_df['mutation_type'] == 'deletion'), ['CHROM', 'POS']
].progress_apply(
    lambda row: get_nucleotide_from_reference(row['CHROM'], int(row['POS']) - 1),
    axis=1
)

# Duplication
cosmic_df.loc[cosmic_df['mutation_type'] == 'duplication', 'original_nucleotide'] = cosmic_df.loc[cosmic_df['ID'].str.contains('dup', na=False), 'ALT'].str[-1]

# deal with start of 1, insertion
cosmic_df.loc[
    (cosmic_df['GENOME_END_POS'].astype(int) == 1) & (cosmic_df['mutation_type'] == 'insertion'), 'original_nucleotide'
] = cosmic_df.loc[
    (cosmic_df['GENOME_END_POS'].astype(int) == 1) & (cosmic_df['mutation_type'] == 'insertion'), ['CHROM', 'POS']
].progress_apply(
    lambda row: get_nucleotide_from_reference(row['CHROM'], int(row['GENOME_END_POS'])),
    axis=1
)

# deal with start of 1, deletion
cosmic_df.loc[
    (cosmic_df['POS'].astype(int) == 1) & (cosmic_df['mutation_type'] == 'deletion'), 'original_nucleotide'
] = cosmic_df.loc[
    (cosmic_df['POS'].astype(int) == 1) & (cosmic_df['mutation_type'] == 'deletion'), ['CHROM', 'POS']
].progress_apply(
    lambda row: get_nucleotide_from_reference(row['CHROM'], int(row['GENOME_END_POS']) + 1),
    axis=1
)

# # deal with (-) strand - commented out because the vcf should all be relative to the forward strand, not the cdna
# cosmic_df.loc[cosmic_df['strand'] == '-', 'original_nucleotide'] = cosmic_df.loc[cosmic_df['strand'] == '-', 'original_nucleotide'].apply(get_complement)

1it [00:00, 246.94it/s]
100%|██████████| 7/7 [00:00<00:00, 1920.59it/s]
1it [00:00, 1161.54it/s]
1it [00:00, 1250.91it/s]


In [101]:
# ins and dup, starting position not 1
cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) != 1)), 'ref_updated'] = cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) != 1)), 'original_nucleotide']
cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) != 1)), 'alt_updated'] = cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) != 1)), 'original_nucleotide'] + cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) != 1)), 'ALT']

# ins and dup, starting position 1
cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) == 1)), 'ref_updated'] = cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) == 1)), 'original_nucleotide']
cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) == 1)), 'alt_updated'] = cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) == 1)), 'ALT'] + cosmic_df.loc[(((cosmic_df['mutation_type'] == 'insertion') | (cosmic_df['mutation_type'] == 'duplication')) & (cosmic_df['POS'].astype(int) == 1)), 'original_nucleotide']


# del, starting position not 1
cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) != 1)), 'ref_updated'] = cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) != 1)), 'original_nucleotide'] + cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) != 1)), 'REF']
cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) != 1)), 'alt_updated'] = cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) != 1)), 'original_nucleotide']

# del, starting position 1
cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) == 1)), 'ref_updated'] = cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) == 1)), 'REF'] + cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) == 1)), 'original_nucleotide']
cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) == 1)), 'alt_updated'] = cosmic_df.loc[((cosmic_df['mutation_type'] == 'deletion') & (cosmic_df['POS'].astype(int) == 1)), 'original_nucleotide']

In [102]:
cosmic_df['ref_updated'] = cosmic_df['ref_updated'].fillna(cosmic_df['REF'])
cosmic_df['alt_updated'] = cosmic_df['alt_updated'].fillna(cosmic_df['ALT'])
cosmic_df.rename(columns={'ALT': 'alt_cosmic', 'alt_updated': 'ALT', 'REF': 'ref_cosmic', 'ref_updated': 'REF'}, inplace=True)
cosmic_df.drop(columns=["Mutation genome position GRCh37", "GENOME_POS", "GENOME_END_POS", "ACCESSION_NUMBER", "Mutation CDS", "mutation_id", 'ref_cosmic', 'alt_cosmic', 'original_nucleotide', 'mutation_type'], inplace=True)  # 'strand'

In [109]:
num_rows_with_na = cosmic_df.isna().any(axis=1).sum()
assert num_rows_with_na == 0, f"Number of rows with NA values: {num_rows_with_na}"

0

In [ ]:
# load in unique_mcrs_df
unique_mcrs_df = pd.read_csv(unique_mcrs_df_path)
unique_mcrs_df.rename(columns={'TP': 'TP_vk', 'FP': 'FP_vk', 'TN': 'TN_vk', 'FN': 'FN_vk'}, inplace=True)

In [ ]:
#  take the intersection of COSMIC and STAR dfs based on CHROM, POS, REF, ALT - but keep the ID from the COSMIC vcf
mut_cosmic_merged_df = pd.merge(df_mut, cosmic_df, 
                     on=['CHROM', 'POS', 'REF', 'ALT'], 
                     how='inner',
                     suffixes=('_df1', '_df2'))

mut_cosmic_merged_df = mut_cosmic_merged_df.drop(columns=['ID_df1']).rename(columns={'ID_df2': 'ID'})

id_set_mut = set(mut_cosmic_merged_df['ID'])



hap_cosmic_merged_df = pd.merge(df_hap, cosmic_df, 
                     on=['CHROM', 'POS', 'REF', 'ALT'], 
                     how='inner',
                     suffixes=('_df1', '_df2'))

hap_cosmic_merged_df = hap_cosmic_merged_df.drop(columns=['ID_df1']).rename(columns={'ID_df2': 'ID'})

id_set_hap = set(hap_cosmic_merged_df['ID'])

In [ ]:
number_of_mutations_mutect = len(df_mut)
number_of_cosmic_mutations_mutect = len(mut_cosmic_merged_df)
number_of_mutations_haplotypecaller = len(df_hap)
number_of_cosmic_mutations_haplotypecaller = len(hap_cosmic_merged_df)

In [ ]:
unique_mcrs_df['mutation_detected_gatk_mutect2'] = unique_mcrs_df['reference_header'].isin(id_set_mut)  #!!! ensure 'reference_header' is correct here  # keep in mind that my IDs are the mutation headers (ENST...), NOT mcrs headers or mcrs ids

unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_mutect2'])
unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_mutect2'])

mutect_stat_path = f"{gatk_parent}/reference_metrics_mutect2.txt"
metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = False, out = mutect_stat_path)
draw_confusion_matrix(metric_dictionary_reference)

true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['mutation_detected_gatk_mutect2'], 'mcrs_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_cosmic_only_mutect2.png")


noncosmic_mutation_id_set = {f'mutect_fp_{i}' for i in range(1, number_of_mutations_mutect - number_of_cosmic_mutations_mutect + 1)}
positive_set_including_noncosmic_mutations = positive_set.union(noncosmic_mutation_id_set)

FP_including_noncosmic = metric_dictionary_reference['FP'] + len(positive_set_including_noncosmic_mutations)
accuracy, sensitivity, specificity = calculate_sensitivity_specificity(metric_dictionary_reference['TP'], metric_dictionary_reference['TP'], FP_including_noncosmic, metric_dictionary_reference['TP'])

with open(mutect_stat_path, "a") as file:
    file.write(f"FP including non-cosmic: {FP_including_noncosmic}\n")
    file.write(f"accuracy including non-cosmic: {accuracy}\n")
    file.write(f"specificity including non-cosmic: {specificity}\n")

create_venn_diagram(true_set, positive_set_including_noncosmic_mutations, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_including_noncosmics_mutect2.png")

create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, out_path = f"{plot_output_folder}/accuracy_vs_number_of_reads_mutant.png")
#!!! create similar plots for y in {sensitivity, specificity}, and x in {number_of_reads_wt, tumor_purity} and determine cutoffs for which GATK is reliable

unique_mcrs_df.rename(columns={'TP': 'TP_gatk_mutect2', 'FP': 'FP_gatk_mutect2', 'TN': 'TN_gatk_mutect2', 'FN': 'FN_gatk_mutect2'}, inplace=True)

In [ ]:
unique_mcrs_df['mutation_detected_gatk_haplotypecaller'] = unique_mcrs_df['reference_header'].isin(id_set_hap)

unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])
unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['mutation_detected_gatk_haplotypecaller'])

haplotypecaller_stat_path = f"{gatk_parent}/reference_metrics_haplotypecaller.txt"
metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = False, out = haplotypecaller_stat_path)
draw_confusion_matrix(metric_dictionary_reference)

true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['mutation_detected_gatk_haplotypecaller'], 'mcrs_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_cosmic_only_haplotypecaller.png")



noncosmic_mutation_id_set = {f'haplotypecaller_fp_{i}' for i in range(1, number_of_mutations_haplotypecaller - number_of_cosmic_mutations_haplotypecaller + 1)}
positive_set_including_noncosmic_mutations = positive_set.union(noncosmic_mutation_id_set)

FP_including_noncosmic = metric_dictionary_reference['FP'] + len(positive_set_including_noncosmic_mutations)
accuracy, sensitivity, specificity = calculate_sensitivity_specificity(metric_dictionary_reference['TP'], metric_dictionary_reference['TP'], FP_including_noncosmic, metric_dictionary_reference['TP'])

with open(haplotypecaller_stat_path, "a") as file:
    file.write(f"FP including non-cosmic: {FP_including_noncosmic}\n")
    file.write(f"accuracy including non-cosmic: {accuracy}\n")
    file.write(f"specificity including non-cosmic: {specificity}\n")

create_venn_diagram(true_set, positive_set_including_noncosmic_mutations, TN = metric_dictionary_reference['TN'], out_path = f"{gatk_parent}/venn_diagram_reference_including_noncosmics_haplotypecaller.png")

create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, out_path = f"{plot_output_folder}/accuracy_vs_number_of_reads_mutant.png")
#!!! create similar plots for y in {sensitivity, specificity}, and x in {number_of_reads_wt, tumor_purity} and determine cutoffs for which GATK is reliable

unique_mcrs_df.rename(columns={'TP': 'TP_gatk_haplotypecaller', 'FP': 'FP_gatk_haplotypecaller', 'TN': 'TN_gatk_haplotypecaller', 'FN': 'FN_gatk_haplotypecaller'}, inplace=True)